# Importing data from xarray

Written by Jean M Favre, Swiss National Supercomputing Center, to demonstrate the use of the
vtkGridConstructors to import xarray arrays and make a spherical shell

The data is the first step of a time-series provided by https://scivis2021.netlify.app/

Tested on Piz Daint Tue Aug 23 09:28:09 CEST 2022

In [ ]:
from paraview.simple import *
import numpy as np
import sys

# to turn off some annoying warnings
from vtkmodules.vtkCommonCore import vtkLogger
vtkLogger.SetStderrVerbosity(vtkLogger.VERBOSITY_OFF)

sys.path.append("/users/jfavre/Projects/ParaView/Python")
from vtkGridConstructors import vtkStructuredGridFromArrays
from vtk.numpy_interface import algorithms as algs

In [ ]:
view = GetRenderView()
from ipyparaview.widgets import PVDisplay
disp = PVDisplay(view)
w = display(disp)

In [ ]:
import xarray as xr
data = xr.open_dataset('/scratch/snx3000/jfavre/SciVis2021/spherical001.nc')
print(data)

P = (90. - data.lat.values) * np.pi / 180.
R = data.r.values
T = data.lon.values * 2 * np.pi / 360.

p, r, t = np.meshgrid(P, R, T, indexing="ij")
X = r * np.cos(t) * np.sin(p)
Y = r * np.sin(t) * np.sin(p)
Z = r * np.cos(p)

SphericalGrid = vtkStructuredGridFromArrays()

SphericalGrid.SetCoordinates(X, Y, Z)

#add scalar variables
scalar_vars=["temperature",
             "temperature anomaly",
             "thermal conductivity",
             "thermal expansivity",
             "spin transition-induced density anomaly"]

for var in scalar_vars:
  SphericalGrid.AddArray(data.data_vars[var].values.ravel(), var)

# Add velocity (rescaled) as a vector
SphericalGrid.AddArray(1.0e9*algs.make_vector(data.vx.values.ravel(),
                                              data.vy.values.ravel(),
                                              data.vz.values.ravel()), "velocity")


print("StructuredGrid mesh dimensions = ", SphericalGrid.GetDimensions())
print("StructuredGrid mesh bounds = ", SphericalGrid.GetOutput().GetBounds())

pVTrivialProducer2 = PVTrivialProducer()
obj = pVTrivialProducer2.GetClientSideObject()
obj.SetOutput(SphericalGrid.GetOutput())
pVTrivialProducer2.UpdatePipeline()

rep2 = Show(pVTrivialProducer2)
rep2.Representation = 'Surface With Edges'

In [ ]:
ColorBy(rep2, "temperature")

In [ ]:
ResetCamera()
GetActiveCamera().Dolly(2)

In [ ]:
rep2.Representation="Surface"

In [ ]:
Hide(pVTrivialProducer2)

In [ ]:
extractSubset1 = ExtractSubset(Input=pVTrivialProducer2)
extractSubset1.VOI = [0, 270, 0, 200, 0, 179]
# only look at 270 degrees of longitude

rep3 = Show(extractSubset1)
rep3.Representation='Surface'
ColorBy(rep3, "temperature")
